In [1]:
from stim import PauliString
from ucc_ft.checker import ft_check

Detected IPython. Loading juliacall extension. See https://juliapy.github.io/PythonCall.jl/stable/compat/#IPython


This notebook demonstrates finding a circuit error on a small example. In this case, we consider a cat state preparation circuit. Although not a gadget for an error correcting code, we can still definite its fault-tolerance up to an allowed number of faults in the prepared state. 

In [2]:
class CatStateStabilizer:
    def __init__(self, num_qubits: int, max_faults: int):
        self.num_qubits = num_qubits
        self.d = max_faults * 2 + 1

    def stabilizers(self):
        return [
            PauliString(f"Z{i}*Z{j}")
            for (i, j) in zip(range(self.num_qubits), range(1, self.num_qubits))
        ]

    def logical_prep_stabilizer(self):
        """The prepared state is |+>_L, the +1 eigenstate of the logical X operator."""
        return PauliString("X" * self.num_qubits)

    def physical_z_stabilizers(self):
        return [PauliString(f"Z{i}") for i in range(self.num_qubits)]

In [5]:
num_qubits = 4
max_faults = 2
code = CatStateStabilizer(num_qubits, max_faults)

circuit = """
OPENQASM 3.0;
include "stdgates.inc";

const uint size = __NUM_QUBITS__;
qubit[size] state;
qubit ancilla;

def cat_prep() {
    bit res = 1;
    while(res != 0) {
        reset state[0];
        res = 0;
        h state[0];

        // QASM ranges are inclusive for both start and end
        for int i in [1:(size-1)] {
            reset state[i];
            cx state[0], state[i];
        }

        // Parity check
        reset ancilla;
        for int i in [1:(size-1)] {
            cx state[i-1], ancilla;
            cx state[i], ancilla;
        }
        res = measure ancilla;
    }
}
""".replace("__NUM_QUBITS__", str(num_qubits))

res = ft_check(code, circuit, "cat_prep", "prepare", num_ancilla=1)

if res:
    print("The cat state preparation circuit is fault-tolerant.")
else:
    print("The cat state preparation circuit is not fault-tolerant.")

[ Info: '`bitwuzla -rwl 1`' is used as smt solver for FT_condition case
[ Info: '`bitwuzla -rwl 1`' has solved the problem
[ Info: The assignment that generates the bug has been written to ./_temp_check_FT_condition_.output


shape: (4, 8)
shape: (4, 8)
1
>>> 45 INIT target_qubit=1
2
>>> 47 H target_qubit=1
3
>>> 48 INIT target_qubit=2
4
>>> 52 CNOT target_qubit1=1, target_qubit2=2
5
>>> 53 INIT target_qubit=3
6
>>> 57 CNOT target_qubit1=1, target_qubit2=3
7
>>> 58 INIT target_qubit=4
8
>>> 62 CNOT target_qubit1=1, target_qubit2=4
9
>>> 63 INIT target_qubit=5
10
>>> 67 CNOT target_qubit1=1, target_qubit2=5
11
>>> 71 CNOT target_qubit1=2, target_qubit2=5
12
>>> 75 CNOT target_qubit1=2, target_qubit2=5
13
>>> 79 CNOT target_qubit1=3, target_qubit2=5
14
>>> 83 CNOT target_qubit1=3, target_qubit2=5
15
>>> 87 CNOT target_qubit1=4, target_qubit2=5
16
>>> 88 DestructiveM target_qubit=5, sym_name=DestructiveM_cat_prep_5_2
>>> Fail!
The cat state preparation circuit is not fault-tolerant.
